In [30]:
# -*- coding: utf-8 -*-
"""
M_PURCHASE 表探索分析 - 采购单事实表评估
数据治理/数仓建模视角
"""

import sys
import os
# 添加项目路径，以便导入config模块
project_path = r'C:\Users\tianhao\PycharmProjects\hefang_dw'
if project_path not in sys.path:
    sys.path.insert(0, project_path)

import oracledb
import pandas as pd
import numpy as np
from datetime import datetime
from config import ORACLE_CONFIG, ORACLE_DSN

print("="*80)
print("M_PURCHASE 表探索分析 - 采购单事实表评估")
print("="*80)
print(f"分析时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

M_PURCHASE 表探索分析 - 采购单事实表评估
分析时间: 2026-01-16 15:46:22



In [31]:
# 1. 连接Oracle数据库并检查M_PURCHASE表是否存在
print("【步骤1】连接Oracle数据库...")
conn = oracledb.connect(
    user=ORACLE_CONFIG['user'],
    password=ORACLE_CONFIG['password'],
    dsn=ORACLE_DSN
)
print("✓ 连接成功\n")

# 检查M_PURCHASE和M_PURCHASEITEM表是否存在
print("【步骤1.1】检查M_PURCHASE相关表是否存在...")
cursor = conn.cursor()

# 指定架构（schema）
TARGET_SCHEMA = 'BOSNDS3'  # 目标架构
print(f"目标架构: {TARGET_SCHEMA}")

# 步骤1：先在指定架构中查找
print(f"\n步骤1.1.1：在架构 {TARGET_SCHEMA} 中查找M_PURCHASE相关表...")
sql_check_target_schema = f"""
SELECT OWNER, TABLE_NAME
FROM ALL_TABLES
WHERE UPPER(OWNER) = '{TARGET_SCHEMA}'
  AND UPPER(TABLE_NAME) IN ('M_PURCHASE', 'M_PURCHASEITEM')
ORDER BY TABLE_NAME
"""

cursor.execute(sql_check_target_schema)
tables = cursor.fetchall()

if tables:
    print(f"✓ 在架构 {TARGET_SCHEMA} 中找到表：")
    for row in tables:
        owner = row[0]
        table_name = row[1]
        print(f"  - {owner}.{table_name}")
    
    # 确定要使用的表名和schema
    m_purchase_table = None
    m_purchaseitem_table = None
    
    for row in tables:
        owner = row[0]
        table_name = row[1]
        if table_name.upper() == 'M_PURCHASE':
            m_purchase_table = f"{owner}.{table_name}"
        elif table_name.upper() == 'M_PURCHASEITEM':
            m_purchaseitem_table = f"{owner}.{table_name}"
    
    print(f"\n将使用表：")
    print(f"  M_PURCHASE: {m_purchase_table}")
    print(f"  M_PURCHASEITEM: {m_purchaseitem_table}")
    
    # 保存表名供后续使用
    M_PURCHASE_TABLE = m_purchase_table.split('.')[1] if '.' in m_purchase_table else m_purchase_table
    M_PURCHASEITEM_TABLE = m_purchaseitem_table.split('.')[1] if '.' in m_purchaseitem_table else m_purchaseitem_table
    M_PURCHASE_SCHEMA = m_purchase_table.split('.')[0] if '.' in m_purchase_table else None
    M_PURCHASEITEM_SCHEMA = m_purchaseitem_table.split('.')[0] if '.' in m_purchaseitem_table else None
    
else:
    print(f"⚠️ 在架构 {TARGET_SCHEMA} 中未找到M_PURCHASE或M_PURCHASEITEM表")
    
    # 步骤2：在所有架构中查找
    print(f"\n步骤1.1.2：在所有架构中查找M_PURCHASE相关表...")
    sql_check_all = """
    SELECT OWNER, TABLE_NAME
    FROM ALL_TABLES
    WHERE UPPER(TABLE_NAME) IN ('M_PURCHASE', 'M_PURCHASEITEM')
    ORDER BY OWNER, TABLE_NAME
    """
    
    cursor.execute(sql_check_all)
    all_tables = cursor.fetchall()
    
    if all_tables:
        print("找到相关表（所有架构）：")
        for row in all_tables:
            owner = row[0]
            table_name = row[1]
            print(f"  - {owner}.{table_name}")
        
        # 优先使用第一个找到的表
        m_purchase_table = None
        m_purchaseitem_table = None
        
        for row in all_tables:
            owner = row[0]
            table_name = row[1]
            if table_name.upper() == 'M_PURCHASE' and m_purchase_table is None:
                m_purchase_table = f"{owner}.{table_name}"
            elif table_name.upper() == 'M_PURCHASEITEM' and m_purchaseitem_table is None:
                m_purchaseitem_table = f"{owner}.{table_name}"
        
        print(f"\n将使用表：")
        print(f"  M_PURCHASE: {m_purchase_table}")
        print(f"  M_PURCHASEITEM: {m_purchaseitem_table}")
        
        M_PURCHASE_TABLE = m_purchase_table.split('.')[1] if '.' in m_purchase_table else m_purchase_table
        M_PURCHASEITEM_TABLE = m_purchaseitem_table.split('.')[1] if '.' in m_purchaseitem_table else m_purchaseitem_table
        M_PURCHASE_SCHEMA = m_purchase_table.split('.')[0] if '.' in m_purchase_table else None
        M_PURCHASEITEM_SCHEMA = m_purchaseitem_table.split('.')[0] if '.' in m_purchaseitem_table else None
        
    else:
        print("⚠️ 在所有架构中都未找到M_PURCHASE或M_PURCHASEITEM表！")
        print("\n尝试查询当前用户下所有包含'PURCHASE'的表名：")
        sql_find_tables = """
        SELECT OWNER, TABLE_NAME
        FROM ALL_TABLES
        WHERE UPPER(TABLE_NAME) LIKE '%PURCHASE%'
        AND OWNER = USER
        ORDER BY TABLE_NAME
        """
        try:
            cursor.execute(sql_find_tables)
            similar_tables = cursor.fetchall()
            if similar_tables:
                print("  找到相似表名：")
                for row in similar_tables[:10]:
                    owner = row[0]
                    table_name = row[1]
                    print(f"    - {table_name}")
            else:
                print("  未找到相似表名")
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 使用默认值（带schema前缀）
        M_PURCHASE_TABLE = 'M_PURCHASE'
        M_PURCHASEITEM_TABLE = 'M_PURCHASEITEM'
        M_PURCHASE_SCHEMA = TARGET_SCHEMA
        M_PURCHASEITEM_SCHEMA = TARGET_SCHEMA

print(f"\n最终使用的表配置：")
print(f"  M_PURCHASE: {M_PURCHASE_SCHEMA}.{M_PURCHASE_TABLE}" if M_PURCHASE_SCHEMA else f"  M_PURCHASE: {M_PURCHASE_TABLE}")
print(f"  M_PURCHASEITEM: {M_PURCHASEITEM_SCHEMA}.{M_PURCHASEITEM_TABLE}" if M_PURCHASEITEM_SCHEMA else f"  M_PURCHASEITEM: {M_PURCHASEITEM_TABLE}")
print()

【步骤1】连接Oracle数据库...
✓ 连接成功

【步骤1.1】检查M_PURCHASE相关表是否存在...
目标架构: BOSNDS3

步骤1.1.1：在架构 BOSNDS3 中查找M_PURCHASE相关表...
✓ 在架构 BOSNDS3 中找到表：
  - BOSNDS3.M_PURCHASE
  - BOSNDS3.M_PURCHASEITEM

将使用表：
  M_PURCHASE: BOSNDS3.M_PURCHASE
  M_PURCHASEITEM: BOSNDS3.M_PURCHASEITEM

最终使用的表配置：
  M_PURCHASE: BOSNDS3.M_PURCHASE
  M_PURCHASEITEM: BOSNDS3.M_PURCHASEITEM



In [32]:
# 2. 查看M_PURCHASE表结构
print("【步骤2】查看M_PURCHASE表结构...")
print("="*80)

# 查询表结构（使用ALL_TAB_COLUMNS以支持跨schema查询）
if M_PURCHASE_SCHEMA:
    sql_structure = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE,
        DATA_LENGTH,
        DATA_PRECISION,
        DATA_SCALE,
        NULLABLE,
        DATA_DEFAULT
    FROM ALL_TAB_COLUMNS
    WHERE OWNER = '{M_PURCHASE_SCHEMA}'
      AND TABLE_NAME = '{M_PURCHASE_TABLE}'
    ORDER BY COLUMN_ID
    """
else:
    sql_structure = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE,
        DATA_LENGTH,
        DATA_PRECISION,
        DATA_SCALE,
        NULLABLE,
        DATA_DEFAULT
    FROM USER_TAB_COLUMNS
    WHERE TABLE_NAME = '{M_PURCHASE_TABLE}'
    ORDER BY COLUMN_ID
    """

try:
    cursor.execute(sql_structure)
    columns_info = cursor.fetchall()
    df_structure = pd.DataFrame(columns_info, columns=[
        '字段名', '数据类型', '长度', '精度', '小数位', '可空', '默认值'
    ])
    
    if len(df_structure) > 0:
        print(f"\nM_PURCHASE表字段结构（共{len(df_structure)}个字段）：")
        print(df_structure.to_string(index=False))
    else:
        print(f"\n⚠️ 未找到表 {M_PURCHASE_TABLE} 的字段信息")
        print("可能原因：")
        print("1. 表不存在")
        print("2. 当前用户没有查询权限")
        print("3. 表名不正确")
except Exception as e:
    print(f"\n❌ 查询表结构失败: {e}")
    print(f"尝试查询的表名: {M_PURCHASE_TABLE}")
    df_structure = pd.DataFrame()

【步骤2】查看M_PURCHASE表结构...

M_PURCHASE表字段结构（共138个字段）：
                字段名     数据类型   长度   精度  小数位 可空                                      默认值
                 ID   NUMBER   22 10.0  0.0  N                                     None
       AD_CLIENT_ID   NUMBER   22 10.0  0.0  Y                                     None
          AD_ORG_ID   NUMBER   22 10.0  0.0  Y                                     None
            OWNERID   NUMBER   22 10.0  0.0  Y                                     None
         MODIFIERID   NUMBER   22 10.0  0.0  Y                                     None
       CREATIONDATE     DATE    7  NaN  NaN  Y                                     None
       MODIFIEDDATE     DATE    7  NaN  NaN  Y                                  sysdate
           ISACTIVE     CHAR    1  NaN  NaN  N                                     'Y' 
              DOCNO VARCHAR2   80  NaN  NaN  Y                                     None
            DOCTYPE     CHAR    3  NaN  NaN  Y                       

In [33]:
# 3. 查看M_PURCHASEITEM表结构（采购明细）
print("\n【步骤3】查看M_PURCHASEITEM表结构...")
print("="*80)

if M_PURCHASEITEM_SCHEMA:
    sql_structure_item = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE,
        DATA_LENGTH,
        DATA_PRECISION,
        DATA_SCALE,
        NULLABLE,
        DATA_DEFAULT
    FROM ALL_TAB_COLUMNS
    WHERE OWNER = '{M_PURCHASEITEM_SCHEMA}'
      AND TABLE_NAME = '{M_PURCHASEITEM_TABLE}'
    ORDER BY COLUMN_ID
    """
else:
    sql_structure_item = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE,
        DATA_LENGTH,
        DATA_PRECISION,
        DATA_SCALE,
        NULLABLE,
        DATA_DEFAULT
    FROM USER_TAB_COLUMNS
    WHERE TABLE_NAME = '{M_PURCHASEITEM_TABLE}'
    ORDER BY COLUMN_ID
    """

try:
    cursor.execute(sql_structure_item)
    columns_info_item = cursor.fetchall()
    df_structure_item = pd.DataFrame(columns_info_item, columns=[
        '字段名', '数据类型', '长度', '精度', '小数位', '可空', '默认值'
    ])
    
    if len(df_structure_item) > 0:
        print(f"\nM_PURCHASEITEM表字段结构（共{len(df_structure_item)}个字段）：")
        print(df_structure_item.to_string(index=False))
    else:
        print(f"\n⚠️ 未找到表 {M_PURCHASEITEM_TABLE} 的字段信息")
except Exception as e:
    print(f"\n❌ 查询表结构失败: {e}")
    print(f"尝试查询的表名: {M_PURCHASEITEM_TABLE}")
    df_structure_item = pd.DataFrame()


【步骤3】查看M_PURCHASEITEM表结构...

M_PURCHASEITEM表字段结构（共89个字段）：
                      字段名     数据类型  长度   精度  小数位 可空     默认值
                       ID   NUMBER  22 10.0  0.0  N    None
             AD_CLIENT_ID   NUMBER  22 10.0  0.0  Y    None
                AD_ORG_ID   NUMBER  22 10.0  0.0  Y    None
                 ISACTIVE     CHAR   1  NaN  NaN  Y     'Y'
             CREATIONDATE     DATE   7  NaN  NaN  Y    None
                  OWNERID   NUMBER  22 10.0  0.0  Y    None
             MODIFIEDDATE     DATE   7  NaN  NaN  Y sysdate
               MODIFIERID   NUMBER  22 10.0  0.0  Y    None
            M_PURCHASE_ID   NUMBER  22 10.0  0.0  Y    None
                  ORDERNO   NUMBER  22 10.0  0.0  Y    None
             M_PRODUCT_ID   NUMBER  22 10.0  0.0  Y    None
M_ATTRIBUTESETINSTANCE_ID   NUMBER  22 10.0  0.0  Y    None
                      QTY   NUMBER  22 10.0  0.0  Y    None
                    QTYIN   NUMBER  22 10.0  0.0  Y       0
                PRICELIST   NUMBER  22 14

In [34]:
# 4. 数据量统计
print("\n【步骤4】数据量统计...")
print("="*80)

# 构建表名（包含schema前缀）
m_purchase_full = f"{M_PURCHASE_SCHEMA}.{M_PURCHASE_TABLE}" if M_PURCHASE_SCHEMA else M_PURCHASE_TABLE
m_purchaseitem_full = f"{M_PURCHASEITEM_SCHEMA}.{M_PURCHASEITEM_TABLE}" if M_PURCHASEITEM_SCHEMA else M_PURCHASEITEM_TABLE

# M_PURCHASE表使用CREATIONDATE，M_PURCHASEITEM表也使用CREATIONDATE
sql_count = f"""
SELECT 
    'M_PURCHASE' AS 表名,
    COUNT(*) AS 总记录数,
    COUNT(DISTINCT ID) AS 唯一主键数,
    COUNT(CASE WHEN ISACTIVE = 'Y' THEN 1 END) AS 有效记录数,
    COUNT(CASE WHEN ISACTIVE = 'N' THEN 1 END) AS 无效记录数,
    MIN(CREATIONDATE) AS 最早创建时间,
    MAX(CREATIONDATE) AS 最新创建时间
FROM {m_purchase_full}
UNION ALL
SELECT 
    'M_PURCHASEITEM' AS 表名,
    COUNT(*) AS 总记录数,
    COUNT(DISTINCT ID) AS 唯一主键数,
    COUNT(CASE WHEN ISACTIVE = 'Y' THEN 1 END) AS 有效记录数,
    COUNT(CASE WHEN ISACTIVE = 'N' THEN 1 END) AS 无效记录数,
    MIN(CREATIONDATE) AS 最早创建时间,
    MAX(CREATIONDATE) AS 最新创建时间
FROM {m_purchaseitem_full}
"""

try:
    cursor.execute(sql_count)
    count_info = cursor.fetchall()
    df_count = pd.DataFrame(count_info, columns=[
        '表名', '总记录数', '唯一主键数', '有效记录数', '无效记录数', '最早创建时间', '最新创建时间'
    ])
    print(df_count.to_string(index=False))
except Exception as e:
    print(f"❌ 查询数据量失败: {e}")
    print(f"尝试查询的表: {m_purchase_full}, {m_purchaseitem_full}")
    print("\n提示：如果表不存在，请检查：")
    print("1. 表名是否正确（注意大小写）")
    print("2. 当前用户是否有查询权限")
    print("3. 表是否在其他schema中")


【步骤4】数据量统计...
            表名   总记录数  唯一主键数  有效记录数  无效记录数              最早创建时间              最新创建时间
    M_PURCHASE  22232  22232  20681   1551 2022-01-05 18:12:34 2026-01-16 14:48:03
M_PURCHASEITEM 118814 118814 118814      0 2022-01-05 18:19:47 2026-01-16 14:48:03


In [35]:
# 5. 查看M_PURCHASE表数据样本（有效记录）
print("\n【步骤5】M_PURCHASE表数据样本（前10条有效记录）...")
print("="*80)

m_purchase_full = f"{M_PURCHASE_SCHEMA}.{M_PURCHASE_TABLE}" if M_PURCHASE_SCHEMA else M_PURCHASE_TABLE

# 使用CREATIONDATE字段排序
sql_sample = f"""
SELECT *
FROM (
    SELECT *
    FROM {m_purchase_full}
    WHERE ISACTIVE = 'Y'
    ORDER BY CREATIONDATE DESC NULLS LAST
)
WHERE ROWNUM <= 10
"""

try:
    cursor.execute(sql_sample)
    columns = [col[0] for col in cursor.description]
    data = cursor.fetchall()
    df_sample = pd.DataFrame(data, columns=columns)
    
    print(f"\n样本数据（共{len(df_sample)}条）：")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 30)
    print(df_sample)
except Exception as e:
    print(f"❌ 查询失败: {e}")
    print(f"尝试查询的表: {m_purchase_full}")


【步骤5】M_PURCHASE表数据样本（前10条有效记录）...

样本数据（共10条）：
      ID  AD_CLIENT_ID  AD_ORG_ID  OWNERID  MODIFIERID        CREATIONDATE  \
0  27008            37         27     1341        1341 2026-01-16 14:48:03   
1  27007            37         27     1341        1341 2026-01-16 14:46:11   
2  27006            37         27     1341        1341 2026-01-16 14:44:59   
3  27005            37         27     1793        1793 2026-01-16 14:21:18   
4  27004            37         27     1793        1793 2026-01-16 14:08:49   
5  27003            37         27     1793        1793 2026-01-16 14:08:17   
6  27002            37         27     1793        1793 2026-01-16 13:52:43   
7  27001            37         27     1897        1897 2026-01-16 11:46:46   
8  27000            37         27     1897        1897 2026-01-16 10:58:08   
9  26999            37         27     1897        1897 2026-01-16 10:57:39   

         MODIFIEDDATE ISACTIVE            DOCNO DOCTYPE  BILLDATE SALESREP_ID  \
0 2026-01-16

In [ ]:
# 7. 探索M_PURCHASE表的"入库线索字段"
print("\n【步骤7】探索M_PURCHASE表的「入库线索字段」...")
print("="*80)

# 查找所有包含"IN"的字段（入库相关）
if M_PURCHASE_SCHEMA:
    sql_in_fields = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE,
        DATA_PRECISION,
        DATA_SCALE,
        NULLABLE
    FROM ALL_TAB_COLUMNS
    WHERE OWNER = '{M_PURCHASE_SCHEMA}'
      AND TABLE_NAME = '{M_PURCHASE_TABLE}'
      AND UPPER(COLUMN_NAME) LIKE '%IN%'
    ORDER BY COLUMN_ID
    """
else:
    sql_in_fields = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE,
        DATA_PRECISION,
        DATA_SCALE,
        NULLABLE
    FROM USER_TAB_COLUMNS
    WHERE TABLE_NAME = '{M_PURCHASE_TABLE}'
      AND UPPER(COLUMN_NAME) LIKE '%IN%'
    ORDER BY COLUMN_ID
    """

try:
    cursor.execute(sql_in_fields)
    in_fields = cursor.fetchall()
    
    if in_fields:
        print("\n发现所有包含'IN'的字段（可能是入库相关）：")
        df_in_fields = pd.DataFrame(in_fields, columns=[
            '字段名', '数据类型', '精度', '小数位', '可空'
        ])
        print(df_in_fields.to_string(index=False))
        
        # 重点分析入库相关字段
        print("\n" + "="*80)
        print("重点入库线索字段分析：")
        print("="*80)
        
        m_purchase_full = f"{M_PURCHASE_SCHEMA}.{M_PURCHASE_TABLE}" if M_PURCHASE_SCHEMA else M_PURCHASE_TABLE
        
        # 分析入库状态字段
        print("\n1. IN_STATUS（入库状态）字段分析：")
        sql_in_status = f"""
        SELECT 
            IN_STATUS,
            COUNT(*) AS 数量,
            ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y'
        GROUP BY IN_STATUS
        ORDER BY IN_STATUS
        """
        try:
            cursor.execute(sql_in_status)
            status_dist = cursor.fetchall()
            if status_dist:
                df_status = pd.DataFrame(status_dist, columns=['入库状态', '数量', '占比(%)'])
                print(df_status.to_string(index=False))
                print("\n参考：通常 1=未入库, 2=已入库")
            else:
                print("  无法统计IN_STATUS分布")
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 分析入库日期字段
        print("\n2. DATEIN（入库日期）字段分析：")
        sql_datein = f"""
        SELECT 
            COUNT(*) AS 总记录数,
            COUNT(DATEIN) AS 有入库日期的记录数,
            COUNT(*) - COUNT(DATEIN) AS 无入库日期的记录数,
            ROUND(COUNT(DATEIN) * 100.0 / COUNT(*), 2) AS 入库日期填充率,
            MIN(DATEIN) AS 最早入库日期,
            MAX(DATEIN) AS 最新入库日期
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y'
        """
        try:
            cursor.execute(sql_datein)
            datein_info = cursor.fetchone()
            if datein_info:
                print(f"  总记录数: {datein_info[0]}")
                print(f"  有入库日期的记录数: {datein_info[1]}")
                print(f"  无入库日期的记录数: {datein_info[2]}")
                print(f"  入库日期填充率: {datein_info[3]}%")
                print(f"  最早入库日期: {datein_info[4]}")
                print(f"  最新入库日期: {datein_info[5]}")
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 分析入库数量字段
        print("\n3. TOT_QTYIN（入库数量）字段分析：")
        sql_qtyin = f"""
        SELECT 
            COUNT(*) AS 总记录数,
            COUNT(CASE WHEN TOT_QTYIN > 0 THEN 1 END) AS 有入库数量的记录数,
            COUNT(CASE WHEN TOT_QTYIN = 0 THEN 1 END) AS 入库数量为0的记录数,
            COUNT(CASE WHEN TOT_QTYIN IS NULL THEN 1 END) AS 入库数量为空的记录数,
            SUM(TOT_QTYIN) AS 总入库数量,
            AVG(TOT_QTYIN) AS 平均入库数量
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y'
        """
        try:
            cursor.execute(sql_qtyin)
            qtyin_info = cursor.fetchone()
            if qtyin_info:
                print(f"  总记录数: {qtyin_info[0]}")
                print(f"  有入库数量的记录数: {qtyin_info[1]}")
                print(f"  入库数量为0的记录数: {qtyin_info[2]}")
                print(f"  入库数量为空的记录数: {qtyin_info[3]}")
                print(f"  总入库数量: {qtyin_info[4]:,.0f}")
                print(f"  平均入库数量: {qtyin_info[5]:,.2f}")
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 分析入库金额字段
        print("\n4. TOT_AMTIN_ACTUAL（入库实际金额）字段分析：")
        sql_amtin = f"""
        SELECT 
            COUNT(*) AS 总记录数,
            COUNT(CASE WHEN TOT_AMTIN_ACTUAL > 0 THEN 1 END) AS 有入库金额的记录数,
            COUNT(CASE WHEN TOT_AMTIN_ACTUAL = 0 THEN 1 END) AS 入库金额为0的记录数,
            SUM(TOT_AMTIN_ACTUAL) AS 总入库金额,
            AVG(TOT_AMTIN_ACTUAL) AS 平均入库金额
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y'
        """
        try:
            cursor.execute(sql_amtin)
            amtin_info = cursor.fetchone()
            if amtin_info:
                print(f"  总记录数: {amtin_info[0]}")
                print(f"  有入库金额的记录数: {amtin_info[1]}")
                print(f"  入库金额为0的记录数: {amtin_info[2]}")
                print(f"  总入库金额: {amtin_info[3]:,.2f}")
                print(f"  平均入库金额: {amtin_info[4]:,.2f}")
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 分析入库人与入库时间
        print("\n5. INERID（入库人ID）和INTIME（入库时间）字段分析：")
        sql_iner = f"""
        SELECT 
            COUNT(*) AS 总记录数,
            COUNT(INERID) AS 有入库人的记录数,
            COUNT(INTIME) AS 有入库时间的记录数,
            COUNT(CASE WHEN INERID IS NOT NULL AND INTIME IS NOT NULL THEN 1 END) AS 同时有入库人和时间的记录数
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y'
        """
        try:
            cursor.execute(sql_iner)
            iner_info = cursor.fetchone()
            if iner_info:
                print(f"  总记录数: {iner_info[0]}")
                print(f"  有入库人的记录数: {iner_info[1]}")
                print(f"  有入库时间的记录数: {iner_info[2]}")
                print(f"  同时有入库人和时间的记录数: {iner_info[3]}")
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 对比采购数量与入库数量
        print("\n6. 采购数量 vs 入库数量对比分析：")
        sql_compare = f"""
        SELECT 
            COUNT(*) AS 总记录数,
            COUNT(CASE WHEN TOT_QTY > 0 THEN 1 END) AS 有采购数量的记录数,
            COUNT(CASE WHEN TOT_QTYIN > 0 THEN 1 END) AS 有入库数量的记录数,
            COUNT(CASE WHEN TOT_QTY = TOT_QTYIN THEN 1 END) AS 完全入库的记录数,
            COUNT(CASE WHEN TOT_QTYIN < TOT_QTY AND TOT_QTYIN > 0 THEN 1 END) AS 部分入库的记录数,
            COUNT(CASE WHEN TOT_QTYIN > TOT_QTY THEN 1 END) AS 超量入库的记录数,
            COUNT(CASE WHEN TOT_QTYIN = 0 OR TOT_QTYIN IS NULL THEN 1 END) AS 未入库的记录数
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y'
          AND TOT_QTY > 0
        """
        try:
            cursor.execute(sql_compare)
            compare_info = cursor.fetchone()
            if compare_info:
                total = compare_info[0]
                print(f"  总记录数（有采购数量）: {total}")
                print(f"  有采购数量的记录数: {compare_info[1]}")
                print(f"  有入库数量的记录数: {compare_info[2]}")
                print(f"  完全入库的记录数: {compare_info[3]} ({compare_info[3]*100.0/total:.2f}%)")
                print(f"  部分入库的记录数: {compare_info[4]} ({compare_info[4]*100.0/total:.2f}%)")
                print(f"  超量入库的记录数: {compare_info[5]} ({compare_info[5]*100.0/total:.2f}%)")
                print(f"  未入库的记录数: {compare_info[6]} ({compare_info[6]*100.0/total:.2f}%)")
        except Exception as e:
            print(f"  查询失败: {e}")
            
    else:
        print("\n⚠️ 未发现包含'IN'的字段")
        
except Exception as e:
    print(f"\n❌ 查询失败: {e}")


【步骤7】探索M_PURCHASE表的「入库线索字段」...

发现所有包含'IN'的字段（可能是入库相关）：
                字段名     数据类型   精度  小数位 可空
          IN_STATUS   NUMBER  1.0  0.0  Y
          TOT_LINES   NUMBER 18.0  0.0  Y
          TOT_QTYIN   NUMBER 18.0  0.0  Y
     TOT_AMTIN_LIST   NUMBER 18.0  4.0  Y
   TOT_AMTIN_ACTUAL   NUMBER 24.0 10.0  Y
             DATEIN   NUMBER  8.0  0.0  Y
             INERID   NUMBER 10.0  0.0  Y
             INTIME     DATE  NaN  NaN  Y
   TOT_AMTIN_PCHECK   NUMBER 24.0 10.0  Y
  TOT_AMTIN_PRECOST   NUMBER 18.0  4.0  Y
          PREDATEIN   NUMBER  8.0  0.0  Y
         PRINTTIMES   NUMBER 10.0  0.0  Y
       PRINTINTIMES   NUMBER 10.0  0.0  Y
   C_CARGOINTYPE_ID   NUMBER 10.0  0.0  Y
TOT_AMTIN_PCHECKTAX   NUMBER 24.0 10.0  Y
      TOT_QTY_INVED   NUMBER 10.0  0.0  Y
        IN_AU_STATE     CHAR  NaN  NaN  Y
        IN_AU_PI_ID   NUMBER 10.0  0.0  Y
       TOT_QTYINVED   NUMBER 10.0  0.0  Y
       TOT_AMTINVED   NUMBER 24.0 10.0  Y
        Y_INFORM_ID   NUMBER 10.0  0.0  Y
         IS_TOWMSIN

In [ ]:
# 8. 深入分析 - 入库完成度与时间分布
print("\n【步骤8】深入分析 - 入库完成度与时间分布...")
print("="*80)

m_purchase_full = f"{M_PURCHASE_SCHEMA}.{M_PURCHASE_TABLE}" if M_PURCHASE_SCHEMA else M_PURCHASE_TABLE

# 1. 入库完成率分析（按入库状态和数量）
print("\n1. 入库完成率详细分析：")
sql_completion = f"""
SELECT 
    CASE 
        WHEN IN_STATUS = 2 AND TOT_QTYIN > 0 AND TOT_QTY > 0 THEN
            CASE 
                WHEN TOT_QTYIN = TOT_QTY THEN '完全入库'
                WHEN TOT_QTYIN < TOT_QTY THEN '部分入库'
                WHEN TOT_QTYIN > TOT_QTY THEN '超量入库'
                ELSE '异常'
            END
        WHEN IN_STATUS = 1 THEN '未入库'
        WHEN TOT_QTY = 0 THEN '无采购数量'
        ELSE '其他'
    END AS 入库状态,
    COUNT(*) AS 数量,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比,
    SUM(TOT_QTY) AS 总采购数量,
    SUM(TOT_QTYIN) AS 总入库数量,
    ROUND(SUM(TOT_QTYIN) * 100.0 / NULLIF(SUM(TOT_QTY), 0), 2) AS 入库完成率
FROM {m_purchase_full}
WHERE ISACTIVE = 'Y'
GROUP BY 
    CASE 
        WHEN IN_STATUS = 2 AND TOT_QTYIN > 0 AND TOT_QTY > 0 THEN
            CASE 
                WHEN TOT_QTYIN = TOT_QTY THEN '完全入库'
                WHEN TOT_QTYIN < TOT_QTY THEN '部分入库'
                WHEN TOT_QTYIN > TOT_QTY THEN '超量入库'
                ELSE '异常'
            END
        WHEN IN_STATUS = 1 THEN '未入库'
        WHEN TOT_QTY = 0 THEN '无采购数量'
        ELSE '其他'
    END
ORDER BY 数量 DESC
"""

try:
    cursor.execute(sql_completion)
    completion_data = cursor.fetchall()
    if completion_data:
        df_completion = pd.DataFrame(completion_data, columns=[
            '入库状态', '数量', '占比(%)', '总采购数量', '总入库数量', '入库完成率(%)'
        ])
        print(df_completion.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")

# 2. 按年月统计入库情况
print("\n2. 按年月统计入库情况（基于DATEIN）：")
sql_monthly = f"""
SELECT 
    TO_CHAR(TO_DATE(TO_CHAR(DATEIN), 'YYYYMMDD'), 'YYYY-MM') AS 年月,
    COUNT(*) AS 入库单数,
    SUM(TOT_QTY) AS 采购数量,
    SUM(TOT_QTYIN) AS 入库数量,
    SUM(TOT_AMT_ACTUAL) AS 采购金额,
    SUM(TOT_AMTIN_ACTUAL) AS 入库金额
FROM {m_purchase_full}
WHERE ISACTIVE = 'Y'
  AND DATEIN IS NOT NULL
  AND DATEIN > 0
GROUP BY TO_CHAR(TO_DATE(TO_CHAR(DATEIN), 'YYYYMMDD'), 'YYYY-MM')
ORDER BY 年月 DESC
"""

try:
    cursor.execute(sql_monthly)
    monthly_data = cursor.fetchall()
    if monthly_data:
        df_monthly = pd.DataFrame(monthly_data, columns=[
            '年月', '入库单数', '采购数量', '入库数量', '采购金额', '入库金额'
        ])
        print(df_monthly.head(12).to_string(index=False))  # 显示最近12个月
        if len(monthly_data) > 12:
            print(f"\n  ... 共{len(monthly_data)}个月的数据，仅显示最近12个月")
except Exception as e:
    print(f"  查询失败: {e}")

# 3. 按供应商统计入库情况
print("\n3. 按供应商统计入库情况（Top 10）：")
sql_supplier = f"""
SELECT 
    mp.C_SUPPLIER_ID AS 供应商ID,
    COUNT(*) AS 采购单数,
    SUM(mp.TOT_QTY) AS 总采购数量,
    SUM(mp.TOT_QTYIN) AS 总入库数量,
    ROUND(SUM(mp.TOT_QTYIN) * 100.0 / NULLIF(SUM(mp.TOT_QTY), 0), 2) AS 入库完成率,
    SUM(mp.TOT_AMTIN_ACTUAL) AS 总入库金额
FROM {m_purchase_full} mp
WHERE mp.ISACTIVE = 'Y'
  AND mp.C_SUPPLIER_ID IS NOT NULL
GROUP BY mp.C_SUPPLIER_ID
ORDER BY 总入库金额 DESC
"""

try:
    cursor.execute(sql_supplier)
    supplier_data = cursor.fetchall()
    if supplier_data:
        df_supplier = pd.DataFrame(supplier_data[:10], columns=[
            '供应商ID', '采购单数', '总采购数量', '总入库数量', '入库完成率(%)', '总入库金额'
        ])
        print(df_supplier.to_string(index=False))
        if len(supplier_data) > 10:
            print(f"\n  ... 共{len(supplier_data)}个供应商，仅显示Top 10")
except Exception as e:
    print(f"  查询失败: {e}")

# 4. 按仓库统计入库情况
print("\n4. 按仓库统计入库情况（Top 10）：")
sql_store = f"""
SELECT 
    mp.C_STORE_ID AS 仓库ID,
    COUNT(*) AS 采购单数,
    SUM(mp.TOT_QTY) AS 总采购数量,
    SUM(mp.TOT_QTYIN) AS 总入库数量,
    ROUND(SUM(mp.TOT_QTYIN) * 100.0 / NULLIF(SUM(mp.TOT_QTY), 0), 2) AS 入库完成率,
    SUM(mp.TOT_AMTIN_ACTUAL) AS 总入库金额
FROM {m_purchase_full} mp
WHERE mp.ISACTIVE = 'Y'
  AND mp.C_STORE_ID IS NOT NULL
GROUP BY mp.C_STORE_ID
ORDER BY 总入库金额 DESC
"""

try:
    cursor.execute(sql_store)
    store_data = cursor.fetchall()
    if store_data:
        df_store = pd.DataFrame(store_data[:10], columns=[
            '仓库ID', '采购单数', '总采购数量', '总入库数量', '入库完成率(%)', '总入库金额'
        ])
        print(df_store.to_string(index=False))
        if len(store_data) > 10:
            print(f"\n  ... 共{len(store_data)}个仓库，仅显示Top 10")
except Exception as e:
    print(f"  查询失败: {e}")

In [ ]:
# 9. M_PURCHASEITEM明细表的入库分析
print("\n【步骤9】M_PURCHASEITEM明细表的入库分析...")
print("="*80)

m_purchaseitem_full = f"{M_PURCHASEITEM_SCHEMA}.{M_PURCHASEITEM_TABLE}" if M_PURCHASEITEM_SCHEMA else M_PURCHASEITEM_TABLE

# 1. 明细级别的入库数量分析
print("\n1. 明细级别的入库数量（QTYIN）分析：")
sql_item_qtyin = f"""
SELECT 
    COUNT(*) AS 总明细数,
    COUNT(CASE WHEN QTYIN > 0 THEN 1 END) AS 有入库数量的明细数,
    COUNT(CASE WHEN QTYIN = 0 THEN 1 END) AS 入库数量为0的明细数,
    COUNT(CASE WHEN QTYIN IS NULL THEN 1 END) AS 入库数量为空的明细数,
    SUM(QTY) AS 总采购数量,
    SUM(QTYIN) AS 总入库数量,
    ROUND(SUM(QTYIN) * 100.0 / NULLIF(SUM(QTY), 0), 2) AS 明细入库完成率,
    AVG(QTY) AS 平均采购数量,
    AVG(QTYIN) AS 平均入库数量
FROM {m_purchaseitem_full}
WHERE ISACTIVE = 'Y'
"""

try:
    cursor.execute(sql_item_qtyin)
    item_qtyin_info = cursor.fetchone()
    if item_qtyin_info:
        print(f"  总明细数: {item_qtyin_info[0]:,}")
        print(f"  有入库数量的明细数: {item_qtyin_info[1]:,}")
        print(f"  入库数量为0的明细数: {item_qtyin_info[2]:,}")
        print(f"  入库数量为空的明细数: {item_qtyin_info[3]:,}")
        print(f"  总采购数量: {item_qtyin_info[4]:,.0f}")
        print(f"  总入库数量: {item_qtyin_info[5]:,.0f}")
        print(f"  明细入库完成率: {item_qtyin_info[6]:.2f}%")
        print(f"  平均采购数量: {item_qtyin_info[7]:,.2f}")
        print(f"  平均入库数量: {item_qtyin_info[8]:,.2f}")
except Exception as e:
    print(f"  查询失败: {e}")

# 2. 明细级别的入库完成情况
print("\n2. 明细级别的入库完成情况：")
sql_item_completion = f"""
SELECT 
    CASE 
        WHEN QTY > 0 AND QTYIN = QTY THEN '完全入库'
        WHEN QTY > 0 AND QTYIN > 0 AND QTYIN < QTY THEN '部分入库'
        WHEN QTY > 0 AND QTYIN > QTY THEN '超量入库'
        WHEN QTY > 0 AND (QTYIN = 0 OR QTYIN IS NULL) THEN '未入库'
        WHEN QTY = 0 THEN '无采购数量'
        ELSE '其他'
    END AS 入库状态,
    COUNT(*) AS 明细数,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) AS 占比,
    SUM(QTY) AS 总采购数量,
    SUM(QTYIN) AS 总入库数量
FROM {m_purchaseitem_full}
WHERE ISACTIVE = 'Y'
GROUP BY 
    CASE 
        WHEN QTY > 0 AND QTYIN = QTY THEN '完全入库'
        WHEN QTY > 0 AND QTYIN > 0 AND QTYIN < QTY THEN '部分入库'
        WHEN QTY > 0 AND QTYIN > QTY THEN '超量入库'
        WHEN QTY > 0 AND (QTYIN = 0 OR QTYIN IS NULL) THEN '未入库'
        WHEN QTY = 0 THEN '无采购数量'
        ELSE '其他'
    END
ORDER BY 明细数 DESC
"""

try:
    cursor.execute(sql_item_completion)
    item_completion_data = cursor.fetchall()
    if item_completion_data:
        df_item_completion = pd.DataFrame(item_completion_data, columns=[
            '入库状态', '明细数', '占比(%)', '总采购数量', '总入库数量'
        ])
        print(df_item_completion.to_string(index=False))
except Exception as e:
    print(f"  查询失败: {e}")

# 3. 主表与明细表的入库数量一致性检查
print("\n3. 主表与明细表入库数量一致性检查：")
sql_consistency = f"""
SELECT 
    COUNT(DISTINCT mp.ID) AS 采购单数,
    COUNT(CASE WHEN ABS(mp.TOT_QTYIN - COALESCE(item_sum.TOT_QTYIN, 0)) <= 0.01 THEN 1 END) AS 数量一致的单数,
    COUNT(CASE WHEN ABS(mp.TOT_QTYIN - COALESCE(item_sum.TOT_QTYIN, 0)) > 0.01 THEN 1 END) AS 数量不一致的单数,
    ROUND(COUNT(CASE WHEN ABS(mp.TOT_QTYIN - COALESCE(item_sum.TOT_QTYIN, 0)) <= 0.01 THEN 1 END) * 100.0 / COUNT(DISTINCT mp.ID), 2) AS 一致性比例
FROM {m_purchase_full} mp
LEFT JOIN (
    SELECT 
        M_PURCHASE_ID,
        SUM(QTYIN) AS TOT_QTYIN
    FROM {m_purchaseitem_full}
    WHERE ISACTIVE = 'Y'
    GROUP BY M_PURCHASE_ID
) item_sum ON mp.ID = item_sum.M_PURCHASE_ID
WHERE mp.ISACTIVE = 'Y'
"""

try:
    cursor.execute(sql_consistency)
    consistency_info = cursor.fetchone()
    if consistency_info:
        print(f"  采购单数: {consistency_info[0]:,}")
        print(f"  数量一致的单数: {consistency_info[1]:,}")
        print(f"  数量不一致的单数: {consistency_info[2]:,}")
        print(f"  一致性比例: {consistency_info[3]:.2f}%")
        if consistency_info[2] > 0:
            print(f"\n  ⚠️ 发现{consistency_info[2]}个采购单的主表与明细表入库数量不一致")
except Exception as e:
    print(f"  查询失败: {e}")

In [ ]:
# 10. 综合评估 - M_PURCHASE作为采购入库事实表的可行性
print("\n【步骤10】综合评估 - M_PURCHASE作为采购入库事实表的可行性...")
print("="*80)
print("\n" + "="*80)
print("📊 M_PURCHASE表作为「采购入库事实表」的评估报告")
print("="*80)

m_purchase_full = f"{M_PURCHASE_SCHEMA}.{M_PURCHASE_TABLE}" if M_PURCHASE_SCHEMA else M_PURCHASE_TABLE

assessments = []

# 1. 表结构评估
assessments.append("✅ 表结构：主表(M_PURCHASE) + 明细表(M_PURCHASEITEM) 结构合理")

# 2. 入库线索字段完整性
try:
    cursor.execute(f"""
        SELECT COUNT(*) FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y' AND DATEIN IS NOT NULL
    """)
    datein_count = cursor.fetchone()[0]
    cursor.execute(f"SELECT COUNT(*) FROM {m_purchase_full} WHERE ISACTIVE = 'Y'")
    total_count = cursor.fetchone()[0]
    fill_rate = datein_count * 100.0 / total_count if total_count > 0 else 0
    if fill_rate >= 95:
        assessments.append(f"✅ 入库日期字段：填充率{fill_rate:.2f}%，完整性良好")
    elif fill_rate >= 80:
        assessments.append(f"⚠️ 入库日期字段：填充率{fill_rate:.2f}%，需要关注")
    else:
        assessments.append(f"❌ 入库日期字段：填充率{fill_rate:.2f}%，完整性不足")
except:
    assessments.append("⚠️ 入库日期字段：无法评估")

# 3. 入库状态字段
try:
    cursor.execute(f"""
        SELECT COUNT(*) FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y' AND IN_STATUS = 2
    """)
    in_status_count = cursor.fetchone()[0]
    cursor.execute(f"SELECT COUNT(*) FROM {m_purchase_full} WHERE ISACTIVE = 'Y'")
    total_count = cursor.fetchone()[0]
    in_rate = in_status_count * 100.0 / total_count if total_count > 0 else 0
    assessments.append(f"✅ 入库状态字段：{in_status_count}/{total_count} ({in_rate:.2f}%)已入库")
except:
    assessments.append("⚠️ 入库状态字段：无法评估")

# 4. 入库数量字段
try:
    cursor.execute(f"""
        SELECT 
            COUNT(CASE WHEN TOT_QTYIN > 0 THEN 1 END),
            COUNT(*)
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y'
    """)
    qtyin_info = cursor.fetchone()
    if qtyin_info:
        qtyin_rate = qtyin_info[0] * 100.0 / qtyin_info[1] if qtyin_info[1] > 0 else 0
        assessments.append(f"✅ 入库数量字段：{qtyin_info[0]}/{qtyin_info[1]} ({qtyin_rate:.2f}%)有入库数量")
except:
    assessments.append("⚠️ 入库数量字段：无法评估")

# 5. 维度完整性
dimension_fields = ['C_STORE_ID', 'C_SUPPLIER_ID', 'BILLDATE', 'DATEIN']
dimension_count = 0
for field in dimension_fields:
    try:
        cursor.execute(f"""
            SELECT COUNT(*) FROM {m_purchase_full}
            WHERE ISACTIVE = 'Y' AND {field} IS NOT NULL
        """)
        count = cursor.fetchone()[0]
        if count > 0:
            dimension_count += 1
    except:
        pass

if dimension_count >= 3:
    assessments.append(f"✅ 维度字段：发现{dimension_count}/4个关键维度字段（仓库、供应商、日期）")
else:
    assessments.append(f"⚠️ 维度字段：仅发现{dimension_count}/4个关键维度字段")

# 6. 度量字段
measure_fields = ['TOT_QTYIN', 'TOT_AMTIN_ACTUAL']
measure_count = 0
for field in measure_fields:
    try:
        cursor.execute(f"""
            SELECT COUNT(*) FROM {m_purchase_full}
            WHERE ISACTIVE = 'Y' AND {field} IS NOT NULL AND {field} > 0
        """)
        count = cursor.fetchone()[0]
        if count > 0:
            measure_count += 1
    except:
        pass

if measure_count >= 2:
    assessments.append(f"✅ 度量字段：发现{measure_count}/2个关键度量字段（数量、金额）")
else:
    assessments.append(f"⚠️ 度量字段：仅发现{measure_count}/2个关键度量字段")

# 7. 数据质量
try:
    cursor.execute(f"""
        SELECT 
            COUNT(CASE WHEN TOT_QTY = TOT_QTYIN THEN 1 END),
            COUNT(*)
        FROM {m_purchase_full}
        WHERE ISACTIVE = 'Y' AND TOT_QTY > 0
    """)
    quality_info = cursor.fetchone()
    if quality_info:
        quality_rate = quality_info[0] * 100.0 / quality_info[1] if quality_info[1] > 0 else 0
        if quality_rate >= 80:
            assessments.append(f"✅ 数据质量：{quality_rate:.2f}%的采购单完全入库，质量良好")
        else:
            assessments.append(f"⚠️ 数据质量：仅{quality_rate:.2f}%的采购单完全入库，需关注部分入库情况")
except:
    assessments.append("⚠️ 数据质量：无法评估")

print("\n评估结果：")
for i, assessment in enumerate(assessments, 1):
    print(f"{i}. {assessment}")

print("\n" + "="*80)
print("💡 结论与建议：")
print("="*80)
print("1. ✅ M_PURCHASE表包含完整的入库线索字段（IN_STATUS, DATEIN, TOT_QTYIN等）")
print("2. ✅ 主表与明细表结构清晰，可以支持不同粒度的分析")
print("3. ✅ 入库日期、入库数量、入库金额等关键字段填充率较高")
print("4. ⚠️ 建议：")
print("   - 使用IN_STATUS=2筛选已入库的采购单作为入库事实")
print("   - 使用DATEIN作为业务日期维度")
print("   - 使用TOT_QTYIN和TOT_AMTIN_ACTUAL作为度量字段")
print("   - 注意处理部分入库的情况（TOT_QTYIN < TOT_QTY）")
print("   - 明细表M_PURCHASEITEM的QTYIN字段可用于SKU级别的入库分析")
print("="*80)

In [37]:
# 6. 关键字段分析 - 维度字段识别
print("\n【步骤6】关键字段分析 - 维度字段识别...")
print("="*80)

# 检查是否有日期字段
if M_PURCHASE_SCHEMA:
    sql_date_fields = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE
    FROM ALL_TAB_COLUMNS
    WHERE OWNER = '{M_PURCHASE_SCHEMA}'
      AND TABLE_NAME = '{M_PURCHASE_TABLE}'
      AND (UPPER(COLUMN_NAME) LIKE '%DATE%' 
           OR UPPER(COLUMN_NAME) LIKE '%TIME%'
           OR UPPER(COLUMN_NAME) LIKE '%BILL%')
    ORDER BY COLUMN_ID
    """
else:
    sql_date_fields = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE
    FROM USER_TAB_COLUMNS
    WHERE TABLE_NAME = '{M_PURCHASE_TABLE}'
    AND (UPPER(COLUMN_NAME) LIKE '%DATE%' 
         OR UPPER(COLUMN_NAME) LIKE '%TIME%'
         OR UPPER(COLUMN_NAME) LIKE '%BILL%')
    ORDER BY COLUMN_ID
    """

try:
    cursor.execute(sql_date_fields)
    date_fields = cursor.fetchall()
    if date_fields:
        print("\n时间维度字段：")
        for field in date_fields:
            print(f"  - {field[0]} ({field[1]})")
    else:
        print("\n未发现明显的时间维度字段")
except Exception as e:
    print(f"\n查询时间字段失败: {e}")

# 检查外键字段（关联维度表）
print("\n外键字段（关联维度表）：")
if M_PURCHASE_SCHEMA:
    sql_fk = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE
    FROM ALL_TAB_COLUMNS
    WHERE OWNER = '{M_PURCHASE_SCHEMA}'
      AND TABLE_NAME = '{M_PURCHASE_TABLE}'
      AND (UPPER(COLUMN_NAME) LIKE '%_ID' 
           OR UPPER(COLUMN_NAME) LIKE '%ID')
    ORDER BY COLUMN_ID
    """
else:
    sql_fk = f"""
    SELECT 
        COLUMN_NAME,
        DATA_TYPE
    FROM USER_TAB_COLUMNS
    WHERE TABLE_NAME = '{M_PURCHASE_TABLE}'
    AND (UPPER(COLUMN_NAME) LIKE '%_ID' 
         OR UPPER(COLUMN_NAME) LIKE '%ID')
    ORDER BY COLUMN_ID
    """

try:
    cursor.execute(sql_fk)
    fk_fields = cursor.fetchall()
    for field in fk_fields:
        print(f"  - {field[0]} ({field[1]})")
except Exception as e:
    print(f"查询外键字段失败: {e}")


【步骤6】关键字段分析 - 维度字段识别...

时间维度字段：
  - CREATIONDATE (DATE)
  - MODIFIEDDATE (DATE)
  - BILLDATE (NUMBER)
  - DATEOUT (NUMBER)
  - DATEIN (NUMBER)
  - STATUSTIME (DATE)
  - INTIME (DATE)
  - PCHECKTIME (DATE)
  - PREDATEIN (NUMBER)
  - PRINTTIMES (NUMBER)
  - PRINTINTIMES (NUMBER)
  - REALDATE (NUMBER)
  - CHECKTIME (DATE)
  - DATEWH (NUMBER)
  - DELIVERYTIME (VARCHAR2)
  - RCOPYTIME (DATE)
  - NC_STATUSTIME (DATE)
  - PRINTTIMES_STATUS (VARCHAR2)
  - PRINTTIMES_DATE (DATE)

外键字段（关联维度表）：
  - ID (NUMBER)
  - AD_CLIENT_ID (NUMBER)
  - AD_ORG_ID (NUMBER)
  - OWNERID (NUMBER)
  - MODIFIERID (NUMBER)
  - SALESREP_ID (NUMBER)
  - C_STORE_ID (NUMBER)
  - C_SUPPLIER_ID (NUMBER)
  - AU_PI_ID (NUMBER)
  - C_PERIOD_ID (NUMBER)
  - B_PO_ID (NUMBER)
  - STATUSERID (NUMBER)
  - INERID (NUMBER)
  - PCHECKID (NUMBER)
  - C_PURCHASETYPE_ID (NUMBER)
  - C_CUSTOMER_ID (NUMBER)
  - OPERATOERID (NUMBER)
  - C_DEPARTMENT_ID (NUMBER)
  - C_CARGOTYPE_ID (NUMBER)
  - C_STORE_LOCATION_ID (NUMBER)
  - C_CARGOINTYP